# 第7回課題

## 課題1

要件に従ってclocktimerを作る。まず、コンストラクタとしてhour, minute, seccondを指定する。次に、tickメソッドを定義する。このメソッドは、一度実行されるごとにsecondを一秒進める。もし60秒に到達すればsecondは0に戻りminuteが+1され、minuteが60になればhourを+1してminuteを0に戻す。そしてhourが24になれば0にリセットされる。


In [57]:
class ClockTimer:
    def __init__(self, hour, minute, second):
        self.hour = hour
        self.minute = minute
        self.second = second

    def tick(self):
        self.second += 1
        if self.second == 60:
            self.second = 0
            self.minute += 1
            if self.minute == 60:
                self.minute = 0
                self.hour += 1
                if self.hour == 24:
                    self.hour = 0


In [58]:
# テストケース
timer = ClockTimer(22, 54, 14)
print(timer.hour, timer.minute, timer.second)
timer.tick()
print(timer.hour, timer.minute, timer.second)
for _ in range(45):
    timer.tick()
print(timer.hour, timer.minute, timer.second) 

22 54 14
22 54 15
22 55 0


## 課題2

フィボナッチ数の計算結果をリストmemoに記録し、計算を行う前に照合する処理を実装する。

コンストラクタはmemoとcounterを用意する。counterに渡すインスタンスは外部クラスCounterで定義されている。

メソッドfibでは、memoに計算結果が存在する場合、それを返す。ない場合は資料で既に実装されているFibクラスのfibメソッドを参考に、再帰的な計算を行う。このとき、計算が行われる度にcounter.incrementでカウンターの数を増やし、計算回数を記録する。

In [59]:
class Counter:
    def __init__(self):
        self.__count = 0

    def increment(self):
        self.__count += 1

    def count(self):
        return self.__count

class MemoFib:
    def __init__(self):
        self.__memo = [None] * 1000
        self.__counter = Counter()

    def fib(self, n):
        if n < 2:
            if self.__memo[n] is None:
                self.__counter.increment()
                self.__memo[n] = n
            return self.__memo[n]
        
        if self.__memo[n] is not None:
            return self.__memo[n]
        
        self.__counter.increment()
        result = self.fib(n - 1) + self.fib(n - 2)
        self.__memo[n] = result
        return result

    def count(self):
        return self.__counter.count()

In [60]:
# テストケース
m = MemoFib()
print(m.fib(10)) 
print(m.count())
print(m.fib(20)) 
print(m.count())  
print(m.fib(20))  
print(m.count())  

55
11
6765
21
6765
21


## 課題3

資料の実装を追いかけていく。まず、協力戦略と裏切り戦略を01で定義する。valid_actionはこれ以外の戦略に対してFalseを返す。Rewardは7.2.1で定義された状況から抽出されている。

次にCooperatePlayer, DefectPlayer, RandomPlayerクラスを定義する。いずれのクラスも自身の名前を返すnameメソッドと行動方針を返すplayメソッドを持つ。

そして彼らを戦わせるためにplay_one_gameを定義する。引数にプレイヤーA, Bのインスタンスをとり、それぞれのplayメソッドで行動を決定したあと、valid_actionを経てそれぞれのRewardを決定する。あとはその内容をprintして結果を戻り値にする。

次にこれを繰り返すplay_games関数を作る。
このために、前回相手が選んだ選択肢に応じて、プレイヤーに次の行動を決めさせるメソッドupdateを各プレイヤークラスに作る必要がある。ただし、この三種類のプレイヤーの行動は前回の行動に依存しないので、pass以外の処理はない。
各プレイヤーにupdateメソッドを作ってもいいが、動作が共通しているなら、親としてPlyaerクラスを作り、そこから継承して各プレイヤークラスを作ると簡便である。親クラスのメソッドは子クラスにも継承されるので、子クラス内で再定義されない限り、それを再利用することができる。

ここまでを踏まえて、新しいプレイヤーTitForTatPlayerを作っていく。これは基本的に協調し、相手が裏切ったらこちらも裏切るプレイヤーである。これまでの各Playerクラスと同様に、Playerクラスを継承し、name, play, updateを再定義する。nameはクラス名のままでよく、playはクラス内で定義されるlast_opponent_actionを返す。これはinit関数により実行時に初期化される。updateは、相手の選んだ戦略を受け取り、それと同じものを返す。


In [66]:
import random

Cooperate = 0
Defect = 1

def valid_action(act):
    return act == Cooperate or act == Defect

Reward = [
    [2, 0],
    [3, 1] 
]


class CooperatePlayer(Player):
    def name(self):
        return "CooperatePlayer"

    def play(self):
        return Cooperate

class DefectPlayer(Player):
    def name(self):
        return "DefectPlayer"

    def play(self):
        return Defect

class RandomPlayer(Player):
    def name(self):
        return "RandomPlayer"

    def play(self):
        return Cooperate if random.randrange(2) == 0 else Defect

class Player:
    def name(self):
        pass

    def play(self):
        pass

    def update(self, my_action, op_action):
        pass
    
def play_games(player_a, player_b, P=1.0/8):
    sum_scores = [0, 0] 
    history = ["", ""]
    
    while True:
        act_a, reward_a, act_b, reward_b = play_one_game(player_a, player_b)
        sum_scores[0] += reward_a
        sum_scores[1] += reward_b
        history[0] += str(act_a)
        history[1] += str(act_b)
        
        if random.random() < P:
            break
    
    print("Scores:", sum_scores)
    print("History:", history)

class TitForTatPlayer(Player):
    def __init__(self):
        self.last_opponent_action = Cooperate

    def name(self):
        return "TitForTatPlayer"

    def play(self):
        return self.last_opponent_action

    def update(self, my_action, op_action):
        self.last_opponent_action = op_action

def play_one_game(player_a, player_b):
    act_a = player_a.play()
    act_b = player_b.play()
    if not valid_action(act_a) or not valid_action(act_b):
        raise ValueError("Invalid action")
    reward_a = Reward[act_a][act_b]
    reward_b = Reward[act_b][act_a]
    player_a.update(act_a, act_b)
    player_b.update(act_b, act_a)
    return (act_a, reward_a, act_b, reward_b)


In [67]:
pc = CooperatePlayer()
pd = DefectPlayer()
pr = RandomPlayer()
pt = TitForTatPlayer()

In [68]:
play_games(pc, pr)

Scores: [0, 3]
History: ['0', '1']


In [69]:
print(play_one_game(pc, pd))
print(play_one_game(pc, pr))
print(play_one_game(pd, pr))
print(play_one_game(pt, pr))

(0, 0, 1, 3)
(0, 2, 0, 2)
(1, 3, 0, 0)
(0, 0, 1, 3)


TitForTatPlayerのテストケースを作成する。全部で4パターン存在する。

In [70]:
def test_tftplayer():
    player = TitForTatPlayer()
    act = player.play()
    assert act == Cooperate, "Error: Initial play should be Cooperate."

    player.update(Cooperate, Cooperate)
    act = player.play()
    assert act == Cooperate, "Error: Should continue to Cooperate after opponent Cooperates."
    
    player.update(Cooperate, Defect)
    act = player.play()
    assert act == Defect, "Error: Should Defect after opponent Defects."
    
    player.update(Defect, Cooperate)
    act = player.play()
    assert act == Cooperate, "Error: Should return to Cooperate if opponent returns to Cooperate after Defecting."
    
    player.update(Defect, Defect)
    act = player.play()
    assert act == Defect, "Error: Should return to Cooperate if opponent returns to Cooperate after Defecting."
    
    print("All tests passed!")

test_tftplayer()

All tests passed!
